# Example: Tensor Network Slicing as Blocks

In [32]:
import quimb.tensor as qtn
import cotengra as ctg
import opt_einsum as oe
import math
import numpy as np
import rosnet as rn

In [21]:
n = 100
reg = 3
d_min = 2
d_max = 2
n_out = 0
seed = 0
eq, shapes = oe.helpers.rand_equation(n=n, reg=reg, n_out=out, d_min=d_min, d_max=d_max, seed=seed)
signatures = eq.split("->")[0].split(",")

In [39]:
tensors = [qtn.rand_tensor(shape, inds) for shape, inds in zip(shapes, signatures)]

NameError: name 'signatures' is not defined

In [ ]:
optimizer = "greedy"
path, info = oe.contract_path(eq, *tensors, optimize=optimizer)
print(f"FLOPs: {info.opt_cost}")
print(f"Order of largest intermediate tensor: {math.log2(info.largest_intermediate)}")

FLOPs: 913012600
Order of largest intermediate tensor: 22.0


In [ ]:
cut_slices = 32
sf = ctg.SliceFinder(info, target_slices=cut_slices, temperature=0)
ix_sl, cost_sl = sf.search()
print(cost_sl)

<ContractionCosts(flops=9.539e+08, size=1.311e+05, nslices=3.200e+01, overhead=1.045)>


In [40]:
for tensor in tensors:
	if any(label in ix_sl for label in tensor.inds):
		print("AAA")
		blockshape = tuple(map(
			lambda x: 1 if x[1] else tensor.shape[x[0]],
			enumerate(label in ix_sl for label in tensor.inds),
		))
	else:
		blockshape = tensor.data.shape

	blocks = np.split(tensor.data, (blockshape))
	tensor.modify(data=rn.BlockArray(tensor.data))

AAA


NotImplementedError: 